# Upscale testing

In [1]:
from PIL import Image
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
from helper_functions.helper_functions import pixels_between_points
import pathlib
import platform
plt2 = platform.system()
if plt2 == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

from scipy.ndimage import map_coordinates

In [2]:
test_image = np.zeros((10,10))
# Pixels
test_image[2, 2:8] = 1
test_image[7, 2:8] = 1
test_image[2:7, 2] = 1
test_image[2:7, 7] = 1
print(test_image)
# 400 points
x_top = np.linspace(2,7)
y_top = np.array([2 for _ in range(len(x_top))])
x_bottom = np.linspace(7,2)
y_bottom = np.array([7 for _ in range(len(x_top))])

x_right = np.array([7 for _ in range(len(x_top))])
y_right = np.linspace(2,7)

x_left = np.array([2 for _ in range(len(x_top))])
y_left = np.linspace(7,2)

fourier_xs = np.concatenate(( x_left, x_top, x_right, x_bottom,)) +0.001
fourier_ys = np.concatenate(( y_left, y_top, y_right, y_bottom,)) +0.001

# Define the scaling factor 
scale_factor_x = 3
scale_factor_y = 3

fourier_xs_with_offset = fourier_xs*scale_factor_x+scale_factor_x/2-0.5
fourier_ys_with_offset = fourier_ys*scale_factor_y+scale_factor_y/2-0.5

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]




Use the get_pixels_between() on the upscaled image using different border scaled differently. Plot and print image to see which pixels are grabbed.
Visually verify the new pixel border an see if it lines up correctly in the printout.

In [3]:
def gridlines(image: np.ndarray):
    xs = []
    ys = []
    for x in range(image.shape[1]):
        # x=0
        xs += [x-0.5, x-0.5, None]
        ys += [-0.5, image.shape[0]-0.5, None]
    for x in range(image.shape[0]):
        # y=0
        ys += [x-0.5, x-0.5, None]
        xs += [-0.5, image.shape[1]-0.5, None]
    return xs,ys

# Proving the offset is necessary

Granule explorer uses `map_coordinates()` from scipy on granule images. This method treats pixel coordinates as their 'center'. Therfore we need to add the offset when scaling the border.
Write about this in the thesis. Could not find any information about this on the internet??

In [4]:
# map_coordinates() uses pixels as integer centered. This means any value sampled +- 1/2 from the integer coordinate will be the same as the same.
# Had map_coordinates() instead treated them as 'corner integered' a different value would have been returned when we deviate from the coordinate in the minus direction.
assert map_coordinates(test_image, [[1.9],[1.9]], order=0) == np.array([1]), "At this position the value is expected to be 1."
assert map_coordinates(test_image, [[0],[0]], order=0) == np.array([0]), "At this position the value is expected to be 0."

In [5]:
# original_image = Image.fromarray(test_image)

# # Get the size of the original image
# original_width, original_height = original_image.size
# # Calculate the new size of the image after upscaling
# new_width = original_width * scale_factor
# new_height = original_height * scale_factor
# # Resize the image using the new size
# upscaled_image_box = np.array(original_image.resize((new_width, new_height), Image.Resampling.NEAREST))        

# fig = make_subplots(rows=1, cols=1, 
#                     horizontal_spacing=0.01, 
#                     vertical_spacing=0.1,
#                     subplot_titles=('Original image', f"Upscaled image s={scale_factor}"))
# # Add images to plot
# base_image_fig = px.imshow(test_image)
# fig.add_trace(base_image_fig.data[0], 1, 1)
# xs, ys = gridlines(test_image)
# fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

# zi = map_coordinates(original_image, [[1.9],[1.9]], order=0)
# print(zi)
# fig.add_trace(go.Scatter(x=[2,2], y=[7,7], marker=dict(color='red', size=10), name=f"Point"), row=1, col=1)

# fig.update_layout(title_text=f"", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
# fig.update(layout_coloraxis_showscale=False)
# # fig.show()

In [9]:
original_image = Image.fromarray(test_image)

# Get the size of the original image
original_width, original_height = original_image.size
# Calculate the new size of the image after upscaling
new_width = original_width * scale_factor_x
new_height = original_height * scale_factor_y
# Resize the image using the new size
upscaled_image_box = np.array(original_image.resize((new_width, new_height), Image.Resampling.NEAREST))        
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    # subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
                    # subplot_titles=('Original image', f"Image upscaled {scale_factor_x} times")
                    )
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)


fig.add_trace(go.Scatter(x=[2.1,2.1], y=[7.05,7.05], marker=dict(color='cyan', size=10), name=f"Point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[2.1*scale_factor_x+scale_factor_x/2-0.5,2.1*scale_factor_x+scale_factor_x/2-0.5], y=[7.05*scale_factor_y+scale_factor_y/2-0.5, 7.05*scale_factor_y+scale_factor_y/2-0.5], marker=dict(color='cyan', size=10), name=f"Point"), row=1, col=2)


fig.add_annotation(text="A",
                  xref="paper", yref="paper",
                  x=0.005, y=1, showarrow=False)
fig.add_annotation(text="B",
                  xref="paper", yref="paper",
                  x=0.54, y=1, showarrow=False)
fig.update_annotations(font=dict(family="DejaVu Sans", size=40, color="white"))

fig.update_layout(width=1400, height=800, showlegend=False, font_size=25)
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(
    {
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
    }
)
fig.update_layout(    
    {    
        'xaxis':{'range': [-0.5, 9.5]},
        'yaxis':{'range': [-0.5, 9.5]},
        'xaxis2':{'range': [-0.5, 29.5]},
        'yaxis2':{'range': [-0.5, 29.5]},
    })
fig.update_layout(
    font_family="DejaVu Sans",
    font_color="black",
    title_font_family="DejaVu Sans",
    title_font_color="black",
    # legend_title_font_color="green"
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.write_image(file=f"D:/Master/MasterProject/Overleaf_figures/Chapter4/upscale_demo.svg", scale=4)
fig.show()

In [50]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)


fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400p border "), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling when b(point)=point*s", title_x=0.5, width=1400, height=800, showlegend=False, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [51]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x+scale_factor_x/2-0.5, y=fourier_ys*scale_factor_y+scale_factor_y/2-0.5, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s + s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

# Get intersected pixels

In [52]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_2 = upscaled_image_box.copy()
xs, ys = pixels_between_points(fourier_xs*scale_factor_x, fourier_ys*scale_factor_y)
# xs, ys = pixels_between_points(fourier_xs*scale_factor+scale_factor/2-0.5, fourier_ys*scale_factor+scale_factor/2-0.5)
for x,y in list(zip(xs,ys)):
    upscaled_image_box_2[int(y)][int(x)] = 0.5
    
upscaled_image_fig = px.imshow(upscaled_image_box_2)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_2)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with no offset: b(point)=point*s", title_x=0.5, width=1400, height=800, showlegend=False, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

# 

scipy.ndimage import map_cooridnates

In [63]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    # subplot_titles=('Original image', f"Image upscaled {scale_factor_x} times")
                    )
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_3 = upscaled_image_box.copy()
xs_pixel, ys_pixel = pixels_between_points(fourier_xs_with_offset, fourier_ys_with_offset)
# xs, ys = pixels_between_points(fourier_xs*scale_factor+scale_factor/2-0.5, fourier_ys*scale_factor+scale_factor/2-0.5)
for x,y in list(zip(xs_pixel,ys_pixel)):
    upscaled_image_box_3[int(y)][int(x)] = 0.5
    
upscaled_image_fig = px.imshow(upscaled_image_box_3)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_3)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='cyan', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs_with_offset, y=fourier_ys_with_offset, marker=dict(color='cyan', size=10), name=f"400 p border"), row=1, col=2)

fig.add_annotation(text="A",
                  xref="paper", yref="paper",
                  x=0.005, y=1, showarrow=False)
fig.add_annotation(text="B",
                  xref="paper", yref="paper",
                  x=0.54, y=1, showarrow=False)
fig.update_annotations(font=dict(family="DejaVu Sans", size=40, color="white"))

fig.update_layout(width=1400, height=800, showlegend=False, font_size=25)
# fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s+s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=False, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(
    {
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
    }
)
fig.update_layout(    
    {    
        'xaxis':{'range': [-0.5, 9.5]},
        'yaxis':{'range': [-0.5, 9.5]},
        'xaxis2':{'range': [-0.5, 29.5]},
        'yaxis2':{'range': [-0.5, 29.5]},
    })
fig.update_layout(
    font_family="DejaVu Sans",
    font_color="black",
    title_font_family="DejaVu Sans",
    title_font_color="black",
    # legend_title_font_color="green"
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.write_image(file=f"D:/Master/MasterProject/Overleaf_figures/Chapter4/upscale_demo_with_border.svg", scale=4)
fig.show()

In [54]:
# print(test_image)
# print(upscaled_image_box)

In [55]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_3 = upscaled_image_box.copy()
test_x,test_y = np.array([13,19.3]), np.array([19.6,13]) # TODO: Create better implementation. Everything works now? Note the +1/2 to
# TODO: pixels_between_points_2() needs to be pushed by 1/2 to align with 'center pixel' method used by python libraries.
xs_pixel, ys_pixel = pixels_between_points(test_x,test_y) # When given coordinates to this function, add 1/2 to treat pixels as 'corner integer position' and not 'center integered'.
fig.add_trace(go.Scatter(x=test_x, y=test_y, marker=dict(color='cyan', size=10), name=f"Test line"), row=1, col=2)
# xs_pixel, ys_pixel = pixels_between_points_2(fourier_xs*scale_factor, fourier_ys*scale_factor)
for y,x in list(zip(xs_pixel,ys_pixel)):
    try:
        upscaled_image_box_3[int(x)][int(y)] = 0.5 # x goes up, y goes right
    except:
        print("what")
        continue

upscaled_image_fig = px.imshow(upscaled_image_box_3)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_3)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=2)
# fig.add_trace(go.Scatter(x=fourier_xs_with_offset, y=fourier_ys_with_offset, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s+s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=False, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [56]:
def print_matrix(matrix):
    with open('test.txt', 'w+') as f:
        for row in matrix:
            l = ""
            for number in row:
                l += f" {int(number)} "
            f.write(l+"\n")

In [57]:
upscaled_image_box_4 = upscaled_image_box.copy()
# xs_pixel, ys_pixel = pixels_between_points_2(test_x,test_y)
xs_pixel, ys_pixel = pixels_between_points(fourier_xs*scale_factor_x+scale_factor_y/2-1/2, fourier_ys*scale_factor_y+scale_factor_y/2-1/2)
# fig.add_trace(go.Scatter(x=test_x,y=test_y, marker=dict(color='red', size=10), name=f"Test line"), row=1, col=2)
# xs_pixel, ys_pixel = pixels_between_points_2(fourier_xs*scale_factor, fourier_ys*scale_factor)
for x,y in list(zip(xs_pixel,ys_pixel)):
    try:
        upscaled_image_box_4[int(x)][int(y)] = 7 # x goes up, y goes right
    except:
        continue

print_matrix(upscaled_image_box_4)
# print_matrix(test_image)
# with open('test.txt', 'w+') as f:
#     # for row in upscaled_image_box_4:
#     f.writelines(map(str, upscaled_image_box_4))

In [58]:
# fig = make_subplots(rows=1, cols=1, 
#                     horizontal_spacing=0.05, 
#                     vertical_spacing=0.1)
# base_image_fig = px.imshow(test_image)
# fig.add_trace(base_image_fig.data[0], 1, 1)
# fig.update_layout(title_text=f"", title_x=0.5, width=700, height=700, showlegend=True, font_size=11)
# fig.update(layout_coloraxis_showscale=False)
# fig.show()

# Find middle point on line intersecting pixel

In [59]:
original_image = Image.fromarray(test_image)
# Resize the image using the new size
upscaled_image_box = np.array(original_image.resize((new_width, new_height), Image.Resampling.NEAREST))        

start_p = np.array([4,5])
angle = np.deg2rad(56)
scale = 2
end_p = np.array([scale*np.cos(angle)+start_p[0], scale*np.sin(angle)+start_p[1]])

# point on line, maxsomething
on_line = np.array([scale/2 *np.cos(angle)+start_p[0], scale/2 *np.sin(angle)+start_p[1]])
x1 = on_line[0]
y1 = on_line[1]


x_pixel_center = np.floor(x1)
y_pixel_center = np.floor(y1)

h1, h2 = (x_pixel_center+0.5) - x1, (x_pixel_center-0.5) - x1
w1, w2 = (y_pixel_center+0.5) - y1, (y_pixel_center-0.5) - y1
# Top intersection
top_h_diff = np.tan(np.deg2rad(90)-angle)*w1
border_p_top = np.array([x1+top_h_diff, y1+w1])
# Bottom intersection
bottom_h_diff = np.tan(np.deg2rad(90)-angle)*w2
border_p_bottom = np.array([x1+bottom_h_diff, y1+w2])
# Left intersection
left_w_diff = np.tan(angle)*h2
border_p_left = np.array([x1+h2, y1+left_w_diff])
# Right intersection
right_w_diff = np.tan(angle)*h1
border_p_right = np.array([x1+h1, y1+right_w_diff])

possible_points = [border_p_left, border_p_right, border_p_bottom, border_p_top]
possible_points = list(filter(lambda p: x_pixel_center-0.5 <= p[0] <= x_pixel_center+0.5 and y_pixel_center-0.5 <= p[1] <= y_pixel_center+0.5, possible_points))
p1, p2 = list(filter(lambda p: (-1000 < p[0] < 100) and (-100 < p[1] < 1000), possible_points))
middle_point = (p1 + p2) / 2


fig = make_subplots(rows=1, cols=1, 
                    horizontal_spacing=0.01, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image'))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=[start_p[0], end_p[0]], y=[start_p[1], end_p[1]], marker=dict(color='black', size=10), name=f"Point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[on_line[0], on_line[0]], y=[on_line[1], on_line[1]], marker=dict(color='purple', size=10), name=f"Sample point"), row=1, col=1)
# fig.add_trace(go.Scatter(x=[border_p_top[0], border_p_top[0]], y=[border_p_top[1], border_p_top[1]], marker=dict(color='green', size=10), name=f"Top point"), row=1, col=1)
# fig.add_trace(go.Scatter(x=[border_p_bottom[0], border_p_bottom[0]], y=[border_p_bottom[1], border_p_bottom[1]], marker=dict(color='green', size=10), name=f"Bottom point"), row=1, col=1)
# fig.add_trace(go.Scatter(x=[border_p_left[0], border_p_left[0]], y=[border_p_left[1], border_p_left[1]], marker=dict(color='green', size=10), name=f"Left point"), row=1, col=1)
# fig.add_trace(go.Scatter(x=[border_p_right[0], border_p_right[0]], y=[border_p_right[1], border_p_right[1]], marker=dict(color='green', size=10), name=f"Right point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[middle_point[0], middle_point[0]], y=[middle_point[1], middle_point[1]], marker=dict(color='cyan', size=10), name=f"Middle point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[p1[0], p1[0]], y=[p1[1], p1[1]], marker=dict(color='red', size=10), name=f"Border point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[p2[0], p2[0]], y=[p2[1], p2[1]], marker=dict(color='red', size=10), name=f"Border point"), row=1, col=1)

fig.update_layout(title_text=f"", title_x=0.5, width=800, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [60]:
def intersect_point_square(line_point, line_slope, square_side):
    """
    Finds the intersection points between a line and the edges of a square.

    Args:
    line_point: A tuple (x, y) representing a point on the line.
    line_slope: The slope of the line.
    square_side: The length of the square's side.

    Returns:
    A list of intersection points (tuples (x, y)).
    """
    x, y = line_point

    # Define the coordinates of the square's corners
    square_corners = [(0, 0), (square_side, 0), (square_side, square_side), (0, square_side)]
    intersections = []

    # Check each pair of consecutive corners of the square
    for i in range(4):
        x1, y1 = square_corners[i]
        x2, y2 = square_corners[(i + 1) % 4]

        # Check if the line intersects the current edge of the square
        if (x <= max(x1, x2) and x >= min(x1, x2) and
            y <= max(y1, y2) and y >= min(y1, y2)):
            if x1 == x2:  # Vertical line, special case
                intersect_x = x1
                intersect_y = line_slope * (intersect_x - x) + y
            else:
                intersect_x = (y - y1 + line_slope * x - line_slope * x1) / line_slope
                intersect_y = line_slope * (intersect_x - x) + y

            # Check if the intersection point is within the square's boundaries
            if (intersect_x >= 0 and intersect_x <= square_side and
                intersect_y >= 0 and intersect_y <= square_side):
                intersections.append((intersect_x, intersect_y))

    return intersections

# Example usage:
line_point = (2, 1)
line_slope = 0.5  # Example slope
square_side = 5

intersection_points = intersect_point_square(line_point, line_slope, square_side)
print("Intersection points:", intersection_points)


Intersection points: []


In [61]:
def middle_point_line_intersection(radii, angle, x1, y1) -> np.array:    
    x_pixel_center = np.round(x1)
    y_pixel_center = np.round(y1)

    h1, h2 = (x_pixel_center+0.5) - x1, (x_pixel_center-0.5) - x1
    w1, w2 = (y_pixel_center+0.5) - y1, (y_pixel_center-0.5) - y1
    # Top intersection
    top_h_diff = np.tan(np.deg2rad(90)-angle)*w1
    border_p_top = np.array([x1+top_h_diff, y1+w1])
    # Bottom intersection
    bottom_h_diff = np.tan(np.deg2rad(90)-angle)*w2
    border_p_bottom = np.array([x1+bottom_h_diff, y1+w2])
    # Left intersection
    left_w_diff = np.tan(angle)*h2
    border_p_left = np.array([x1+h2, y1+left_w_diff])
    # Right intersection
    right_w_diff = np.tan(angle)*h1
    border_p_right = np.array([x1+h1, y1+right_w_diff])

    possible_points = [border_p_left, border_p_right, border_p_bottom, border_p_top]
    possible_points = list(filter(lambda p: x_pixel_center-0.5 <= p[0] <= x_pixel_center+0.5 and y_pixel_center-0.5 <= p[1] <= y_pixel_center+0.5, possible_points))
    p1, p2 = list(filter(lambda p: (-1000 < p[0] < 100) and (-100 < p[1] < 1000), possible_points))
    middle_point = (p1 + p2) / 2
    return middle_point

# Grab all files in directory

In [62]:
import os
for file in os.listdir("D:\Master\MasterProject\dataset_creation\data\ALL_IMS"):
    if file.endswith(".ims"):
        print(file[:-4])

2020-02-05_14.34.33--NAs--T1354-GFP_Burst
2020-02-05_14.35.36--NAs--T1354-GFP_Burst
2020-02-05_14.36.40--NAs--T1354-GFP_Burst
2020-02-05_14.37.45--NAs--T1354-GFP_Burst
2020-02-05_14.38.46--NAs--T1354-GFP_Burst
2020-02-05_14.39.43--NAs--T1354-GFP_Burst
